# Matching profiles and offers

Objectif: Trouver les profils d'employées qui correspondent le mieux à chaque offre

Exercices:

1. Pour chaque offre, trouver l'employé pour lequel les compétences techniques et les compétences longues correspondent le mieux.

Note: Les fonctions `offer_to_word_doc` et `profile_to_word_doc` dans `utils.py` aident à formatter un profil ou une offre en document Word pour aider à les visualiser.

In [1]:
import re

import nltk
import pandas as pd
from flashtext import KeywordProcessor
from nltk import stem
from os.path import join
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
from unidecode import unidecode
from utils import offer_to_word_doc, profile_to_word_doc

tqdm.pandas()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/philippe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load and clean data
We load all profile data: tech skills and experience descriptions are used to compute similarities, and profile job titles are used to check matching quality. We also load 10 offers for which we want to find the perfect profile.

In [2]:
df_profiles = pd.read_csv('employee_profiles.csv')
df_skills = pd.read_csv('employee_skills.csv')
df_experiences = pd.read_csv('employee_experiences.csv')
df_offers = pd.read_csv('offers.csv')

In [3]:
# Remove profiles with too few skills
n_min_tech_skill = 10
mask_keep_profiles = df_skills.groupby(by="id_profile", sort=False).count() > n_min_tech_skill
selected_profile_ids = mask_keep_profiles[mask_keep_profiles.values].index
df_profiles = df_profiles[df_profiles['id_profile'].isin(selected_profile_ids)]
df_skills = df_skills[df_skills['id_profile'].isin(selected_profile_ids)]
df_experiences = df_experiences[df_experiences['id_profile'].isin(selected_profile_ids)]

In [4]:
# Subsample data: only work with 3000 profiles to speed things up
# n_sub_sample = 3000
# df_profiles = df_profiles.iloc[:n_sub_sample]
# selected_profile_ids = set(df_profiles['id_profile'])
# df_skills = df_skills[df_skills['id_profile'].isin(selected_profile_ids)]
# df_experiences = df_experiences[df_experiences['id_profile'].isin(selected_profile_ids)]

## Text cleaning function
These functions were taken from the two previous notebooks.

In [5]:
def clean_tech_skills(text: str) -> str:
    """Prepares text for keywords extractions."""
    punctuation = ["!", "$", "%", ",", ":", ";", "?", "[", "]", "|"]
    for p in punctuation:
        text = text.replace(p, ' ' + p + ' ')
    to_clean = text.strip()
    to_clean = to_clean.replace('\n', ' ')  # when job_title finishes in next line
    to_clean = re.sub('\s+', ' ', to_clean)  # replacing all spaces characters by one
    to_clean = to_clean.replace(',', ' , ')

    # Case where there is '.' in text:
    if '.' in to_clean:
        if re.findall(r'\D[0-9]+\.[\S]', to_clean) != []:  # cas avec des versions de framework dans les keywords
            to_clean = re.sub(r'\D[0-9]+\.[\S]', '', to_clean)
        if ' . ' in to_clean:
            to_clean = to_clean.replace(' . ', '')
        elif to_clean.endswith('.'):
            to_clean = to_clean[:-1]

    # Case where there is '-' in text
    if '-' in to_clean:
        if re.findall('\w+-\w+', to_clean) != []:
            compound_words = re.findall('\w{4,}-\w{4,}', to_clean)
            for cw in compound_words:
                # Avoid cleaning tech words like ('e-commerce', '')
                to_clean = to_clean.replace(cw, cw.replace('-', ' '))
        else:
            to_clean = to_clean.replace('-', ' ')

    # Plus/hashtag sign cleaned only if it's surrounded by spaces
    to_clean = to_clean.replace(' + ', ' ').replace(' # ', ' ').replace('_', ' ')

    # Replacing apostrophe '’' by '''
    to_clean = re.sub(" *' *", "'", to_clean.replace('’', "'"))

    # Replacing articles with apostrophes
    to_clean = re.sub("d'", "", to_clean)
    to_clean = re.sub("l'", "", to_clean)

    # Replacing /() by spaces
    to_clean = to_clean.replace('/', ' ').replace('(', ' ').replace(')', ' ').replace('* ', '')
    to_clean = to_clean.replace('- ', '').replace(':', '')

    # Case where there is '&' in text
    to_clean = to_clean.replace(' & ', ' et ')

    # cleaning isolated numbers
    to_clean = re.sub(' [0-9] ', '', to_clean)

    cleaned = re.sub('\s+', ' ', to_clean.strip())

    return cleaned.lower()

In [6]:
stemmer = stem.snowball.FrenchStemmer()
stopwords = nltk.corpus.stopwords.words('french')


def remove_stopwords(text: str) -> str:
    """Returns a text with removed stopwords."""
    tt = []
    for word in text.split(' '):
        if word not in stopwords:
            tt.append(word)
    return " ".join(tt).strip()


def stem_sentence(text: str) -> str:
    try:
        text = ' '.join([stemmer.stem(word) for word in remove_stopwords(text.lower()).split(' ')])
        text = unidecode(text)
    except (TypeError, AttributeError):
        return ""
    return text.strip()

## Setup TF-IDF to compute long skill similarities
We will use TF-IDF to compute similarities between experience descriptions (aka. long skills) from profiles and offers. First we fit the TF-IDF using all profile experience descriptions. We then precompute TF-IDF vectors for all profiles so that we don't have to do it for each offer. The result is stored as a sparse CSR matrix.

In [7]:
# Preprocess long skills
df_experiences['stemmed_description'] = df_experiences['description'].progress_apply(stem_sentence)

  0%|          | 0/68231 [00:00<?, ?it/s]

In [8]:
# Fit TF-IDF on stemmed experience descriptions
tfidf_long_skill = TfidfVectorizer()
tfidf_long_skill.fit(df_experiences['stemmed_description'].unique())

# Pre-compute all profile long skill tf idf vectors
experiences_tf_idf = tfidf_long_skill.transform(df_experiences['stemmed_description'])

## Setup TF-IDF to compute tech skill similarities
For simplicity and speed, we will also use TF-IDF to compute similarities between tech skills from profiles and offers. First we fit the TF-IDF using all profile tech skills. We then precompute TF-IDF vectors for all profiles so that we don't have to do it for each offer. The result is stored as a sparse CSR matrix.

In [9]:
# Preprocess tech skills
df_skills['skill'] = df_skills['skill'].apply(clean_tech_skills)
df_offers['clean_description_for_skill'] = df_offers['description'].apply(clean_tech_skills)

In [10]:
# Preprocessing for computing similarities
profiles_skills_df = df_skills.groupby(by='id_profile')['skill'].apply(lambda x: " ".join(x)).reset_index()

# Fit TF-IDF on profile tech skills and pre-compute all profile tech skill tf idf vectors
tfidf_tech_skill = TfidfVectorizer()
tech_skills_tf_idf = tfidf_tech_skill.fit_transform(profiles_skills_df["skill"])

We also prepare a keyword extractor to extract tech skills from offers. It will extract any tech skill found in cleaned profile tech skills.

In [11]:
# Prepare tech skill extractor
tech_skill_finder = KeywordProcessor()
tech_skill_finder.add_keywords_from_list(list(set(df_skills['skill'])))

## Functions to compute similarity between profiles and offers
The two following functions take an offer description (or processed description) as an input as well as a sparse TF-IDF matrix for all profiles. It computes tech/long skill cosine similarity scores between TF-IDF vectors of all profiles and that of the offer.

In [12]:
def get_tech_skill_scores(profiles_skills_df: pd.DataFrame, tech_skills_tf_idf: csr_matrix,
                          offer_clean_description: str) -> pd.DataFrame:
    # Find tech keywords in offer description and convert to tf-idf vector
    tech_skills = tech_skill_finder.extract_keywords(offer_clean_description)
    offer_tf_idf = tfidf_tech_skill.transform([" ".join(tech_skills)])

    # Compute cosine similarity to all profile tech skill lists
    skill_scores = cosine_similarity(tech_skills_tf_idf, offer_tf_idf).reshape(-1)

    # Group experiences back with their profiles, and sum scores
    profile_scores = pd.DataFrame({"id_profile": profiles_skills_df["id_profile"], "tech_score": skill_scores})

    return profile_scores.set_index("id_profile")

In [13]:
def get_long_skill_scores(df_experiences: pd.DataFrame, experiences_tf_idf: csr_matrix,
                          offer_description: str):
    # Stem offer description and convert to tf-idf vector
    stemmed_description = stem_sentence(offer_description)
    offer_tf_idf = tfidf_long_skill.transform([stemmed_description])

    # Compute cosine similarity to all experiences
    experience_scores = cosine_similarity(experiences_tf_idf, offer_tf_idf).reshape(-1)

    # Group experiences back with their profiles, and sum scores
    experience_scores_df = pd.DataFrame({"id_profile": df_experiences["id_profile"], "long_score": experience_scores})
    profile_scores = experience_scores_df.groupby("id_profile", sort=False).median()

    return profile_scores

# Matching 10 offers against all employees
Looping through each offer, we compute tech skill and long skill similarity scores to all profiles. We sum the scores and output the profile that best matches the offer.
Results are displayed below for all 10 offers.

In [14]:
match_dir = "matching"
for i, offer in df_offers.iterrows():
    # Convert offers to Word document
    offer_to_word_doc(*df_offers[["jobtitle", "description", "contract", "company", "location"]].iloc[i],
                      out_file_name=join(match_dir, f"{i}_offre.docx"))
    
    # Tech skill matching
    tech_skill_scores = get_tech_skill_scores(profiles_skills_df, tech_skills_tf_idf,
                                              offer["clean_description_for_skill"])

    # Long skill matching
    long_skill_scores = get_long_skill_scores(df_experiences, experiences_tf_idf, offer["description"])

    # Combine scores
    df_scores = pd.merge(tech_skill_scores, long_skill_scores, left_index=True, right_index=True, how="outer")
    df_scores = df_scores.fillna(0)  # In case a score is missing
    df_scores["global_score"] = df_scores.sum(axis=1)
    best_profile_id = df_scores["global_score"].idxmax().item()
    global_score = df_scores.at[best_profile_id, "global_score"]
    tech_skill_score = df_scores.at[best_profile_id, "tech_score"]
    long_skill_score = df_scores.at[best_profile_id, "long_score"]
    
    # Convert best profile to Word document
    profile_to_word_doc(best_profile_id, df_profiles, df_skills, df_experiences,
                        out_file_name=join(match_dir, f"{i}_candidate.docx"))


    # Best candidate
    print(f"\nMatching results for offer {i} ({offer['jobtitle']}):\n "
          f"\tCandidate #{best_profile_id}\n"
          f"\tScore: {global_score:.3f} (tech_score={tech_skill_score:.3f}, mission_score={long_skill_score:.3f})\n"
          f"\tJob title: {df_profiles.loc[df_profiles['id_profile'] == best_profile_id, 'jobtitle'].item()}\n"
          f"\tSkills: {', '.join(df_skills.loc[df_skills['id_profile'] == best_profile_id, 'skill'])}\n"
          f"\tExperiences: {', '.join(df_experiences.loc[df_experiences['id_profile'] == best_profile_id, 'title'])}\n")


/home/philippe/.anaconda3/envs/mobilite/lib/python3.8/site-packages/docx/styles/styles.py:139: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)



Matching results for offer 0 (developpeur logiciel):
 	Candidate #286255
	Score: 0.342 (tech_score=0.233, mission_score=0.109)
	Job title: responsable de projet
	Skills: project management, agile methodologies, software project management, scrum, public transport, software development, agile project management, change management, information technology, software engineering, software design, software quality assurance, itil, business, informatics, java, c#, sql, php, .net, mysql, javascript, c++ language, oracle, embedded software, html, management, leadership, communication, développement de logiciel, méthodes agiles, gestion de projet, gestion de projet logiciel, intégration, déploiement de logiciel, intégration de données, recherche et développement, gestion équipe, sens de organisation, interoperability, billettique, mvc architecture, gmao, intégration logicielle, méthodologie, gestion des risques, direction
	Experiences: Responsable de développement et d'intégration - Chef de pro